<a href="https://colab.research.google.com/github/kentlundgren/aktieutveckling/blob/main/Vindkraftens_l%C3%B6nsamhet_Nr3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# Importera nödvändiga bibliotek
import subprocess
import sys

def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

try:
    import numpy_financial as npf
except ImportError:
    install('numpy-financial')
    import numpy_financial as npf

import numpy as np

# Funktion för att omvandla decimaltecken från komma till punkt
def convert_decimal(input):
    return float(input.replace(',', '.'))

# Funktion för att fråga efter indata
def ask_input(prompt, example):
    print(f"{prompt} (t.ex. {example}): ", end="")
    return convert_decimal(input())

# Funktion för att skriva ut stora tal med fyra värdesiffror
def format_large_number(num):
    return "{:.4g}".format(num)

# Standardvärden
defaults = {"X": "4", "Y": "12", "Z": "0.42", "P": "25", "Q": "5", "R": "0.1", "S": "42100", "T": "62"}

# Skriv ut standardvärdena
print("Standardvärdena är:")
for key, value in defaults.items():
    print(f"{key} = {value}")

# Fråga användaren om hen vill använda standardvärden
use_defaults = input("Vill du använda standardvärden? (ja/nej): ").lower() == "ja"

# Fråga efter indata
X = convert_decimal(defaults["X"]) if use_defaults else ask_input("Vad är storleken på vindkraftsverket i MW", defaults["X"])
Y = convert_decimal(defaults["Y"]) if use_defaults else ask_input("Vad är investeringsutgiften i Mkr/MW", defaults["Y"])
Z = convert_decimal(defaults["Z"]) if use_defaults else ask_input("Vad är kapacitetsfaktorn", defaults["Z"])
P = int(defaults["P"]) if use_defaults else int(ask_input("Vad är vindkraftverkets livslängd i år", defaults["P"]))
Q = convert_decimal(defaults["Q"]) if use_defaults else ask_input("Vad är kalkylräntan i %", defaults["Q"])
R = convert_decimal(defaults["R"]) if use_defaults else ask_input("Vad är den rörliga driftskostnaden i kr/kWh", defaults["R"])
S = convert_decimal(defaults["S"]) if use_defaults else ask_input("Vad är den fasta driftskostnaden i kr/MW", defaults["S"])
T = convert_decimal(defaults["T"]) if use_defaults else ask_input("Vad är intäkten per kWh producerad el i öre/kWh", defaults["T"])

# Skriv ut de använda värdena
print("\nAnvända värden:")
print(f"X = {X}, Y = {Y}, Z = {Z}, P = {P}, Q = {Q}, R = {R}, S = {S}, T = {T}")

# Beräkna totala investeringskostnaden
total_investment = X * Y * 1e6  # i kr

# Beräkna årlig producerad mängd elenergi i kWh
annual_energy_production = X * 8792 * Z * 1e3  # i kWh

# Beräkna årliga intäkter i kr
annual_revenue = annual_energy_production * T / 100  # i kr

# Beräkna årliga rörliga driftskostnader i kr
annual_variable_costs = annual_energy_production * R  # i kr

# Beräkna årliga fasta driftskostnader i kr
annual_fixed_costs = X * S  # i kr

# Beräkna årliga kostnader i kr
annual_costs = annual_variable_costs + annual_fixed_costs  # i kr

# Beräkna årligt kassaflöde i kr
annual_cash_flow = annual_revenue - annual_costs  # i kr

# Beräkna kapitalkostnaden med hjälp av annuitetsmetoden
annuity_factor = (1 - (1 + Q / 100) ** -P) / (Q / 100)
capital_cost = total_investment / annuity_factor

# Beräkna årligt överskott efter kapitalkostnaden
annual_surplus = annual_cash_flow - capital_cost

# Beräkna nuvärde av kassaflöden under vindkraftverkets livslängd
npv_cash_flows = npf.pv(Q / 100, P, -annual_cash_flow)  # i kr

# Beräkna lönsamheten som nuvärde av kassaflöden minus totala investeringskostnaden
profitability = npv_cash_flows - total_investment  # i kr

# Beräkna internräntan
irr = npf.irr([-total_investment] + [annual_cash_flow]*P)

print(f"\nÖverskott per år: {format_large_number(annual_surplus)} kr")
print(f"Nettonuvärde: {format_large_number(profitability)} kr")
print(f"Internränta: {format_large_number(irr * 100)}%")



Standardvärdena är:
X = 4
Y = 12
Z = 0.42
P = 25
Q = 5
R = 0.1
S = 42100
T = 62
Vill du använda standardvärden? (ja/nej): ja

Använda värden:
X = 4.0, Y = 12.0, Z = 0.42, P = 25, Q = 5.0, R = 0.1, S = 42100.0, T = 62.0

Överskott per år: 4.107e+06 kr
Nettonuvärde: 5.788e+07 kr
Internränta: 15.19%
